In [ ]:
!pip install piano_transcription_inference pretty_midi librosa gradio torch numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=f5b6ee5b18c65b9d1a8de0de5d77ec94e3ed59f427f2680a6d44c07c65dfde2b
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891

In [ ]:
import os
import time
import torch
import librosa
import numpy as np
import pretty_midi
from piano_transcription_inference import PianoTranscription, sample_rate, load_audio
import gradio as gr
import matplotlib.pyplot as plt


In [ ]:
class AMTProcessor:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        print(f"Using device: {self.device}")
        self.transcriptor = PianoTranscription(device=self.device)
        self.sample_rate = sample_rate

    def process_audio(self, audio_path):
        """
        Process audio file and return MIDI path
        """
        # Create output directory if it doesn't exist
        output_dir = os.path.join('/tmp', f'amt_output_{int(time.time())}')
        os.makedirs(output_dir, exist_ok=True)
        print(f"Created output directory: {output_dir}")

        # Load and process audio
        print("Loading audio file...")
        audio, _ = librosa.load(audio_path, sr=self.sample_rate, mono=True)

        # Define output MIDI path
        midi_path = os.path.join(output_dir, 'transcribed_output.mid')

        # Transcribe audio to MIDI
        try:
            print("Transcribing audio to MIDI...")
            transcribed_dict = self.transcriptor.transcribe(audio, midi_path)
            print(f"Successfully transcribed audio to: {midi_path}")
            return midi_path, transcribed_dict
        except Exception as e:
            print(f"Error during transcription: {str(e)}")
            return None, None

    def create_piano_roll(self, midi_path):
        """
        Create piano roll visualization from MIDI file
        """
        try:
            print("Creating piano roll visualization...")
            midi_data = pretty_midi.PrettyMIDI(midi_path)
            piano_roll = midi_data.get_piano_roll()
            return piano_roll
        except Exception as e:
            print(f"Error creating piano roll: {str(e)}")
            return None

In [ ]:
def process_and_visualize(audio_file):
    """
    Main processing function for Gradio interface
    """
    print("Starting audio processing...")

    # Initialize processor
    processor = AMTProcessor()

    # Process audio file - Pass the path directly since it's already a string
    midi_path, transcribed_dict = processor.process_audio(audio_file)  # Remove .name

    if midi_path is None:
        return "Error during transcription", None

    # Create piano roll visualization
    piano_roll = processor.create_piano_roll(midi_path)

    if piano_roll is None:
        return midi_path, "Error creating visualization"

    # Create visualization plot
    plt.figure(figsize=(12, 8))
    plt.imshow(piano_roll, aspect='auto', origin='lower', cmap='hot')
    plt.title('Piano Roll Representation')
    plt.xlabel('Time (frames)')
    plt.ylabel('MIDI Note Number')
    plt.colorbar(label='Velocity')

    # Return both MIDI file and visualization data
    return midi_path, plt.gcf()

In [ ]:
# Cell 5 - Create and Launch Gradio Interface
iface = gr.Interface(
    fn=process_and_visualize,
    inputs=[
        gr.Audio(label="Upload Piano Recording", sources=["upload"], type="filepath")  # Changed source to sources
    ],
    outputs=[
        gr.File(label="Download MIDI File"),
        gr.Plot(label="Piano Roll Visualization")
    ],
    title="AI Piano Tutor - Audio to MIDI Transcription",
    description="""
    Upload a piano recording to convert it into a MIDI file and visualize the transcription.
    The model uses high-resolution piano transcription to accurately capture timing and dynamics.
    """,
    examples=[],
    cache_examples=False,
    theme="default"
)

# Launch the interface
iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/12/01 19:05:44 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: connect: connection refused


<IPython.core.display.Javascript object>